In [1]:
# Copyright 2023 Regeneron Pharmaceuticals Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os
import sys
from pathlib import Path
from os.path import dirname as up

sys.path.append('../distance_based_tools')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
from tcrclustering.cluster.helper import (
    tcrdist_parallel_simple_run,
    tcrdist_simple_run,
    ismart_simple_run
)
from tcrvalid.cluster_loop import *
from tcrvalid.data_subsetting import make_subset,sources_,chains_,features_
from tcrvalid.metrics import get_cluster_purity_df,clustering_scoring
from tcrvalid.defaults import *
from tcrclustering.cluster.prep_gliph_data import *
from tcrvalid.load_models import *
from tcrvalid.physio_embedding import SeqArrayDictConverter

/data/home/allen.leary/.conda/envs/tcrvalid/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(21, 7)
(22, 8)


### Wrapper function for iSMART, tcr-dist and TCR-VALID

In [ ]:
def get_clusterer_by_name(name,df_ref,eps):
    if name=='ismart':
        return ismart_simple_run(df_ref,threshold=eps)
    elif name=='tcrdist':
        kwargs_for_tcrdist={'nproc':8,'min_samples':2,'chains':'B'}
        return tcrdist_parallel_simple_run(df_ref,eps=eps, **kwargs_for_tcrdist)
    elif name=='tcrdist_orig':
        return tcrdist_simple_run(df_ref,chains='B')
    elif name=='tcrvalid':
        mapping = SeqArrayDictConverter()
        from_keras = True #expect True in github code
        model_names = {
                'TRA': ['1_2'],
                'TRB': ['1_2_full_40']
                    }
        loaded_trb_models = load_named_models(model_names['TRB'],chain='TRB',as_keras=from_keras)
        trb_model = loaded_trb_models['1_2_full_40']
        f_l_trb = mapping.seqs_to_array(df_ref.clono_id.values,maxlen=28)
        x_l_trb,_,_ = trb_model.predict(f_l_trb)
        dbs = DBSCAN(eps=eps,min_samples=2,metric='manhattan')
        dbs.fit(x_l_trb) 
        
        #eps_name = str('eps_{'+eps_format+'}').format(eps)
        df_cl=df_ref.copy()
        df_cl['cluster'] = dbs.labels_ # put cluster labels back into df that goes with representations
        
        
        return df_cl
    else:
        raise NotImplementedError("name : "+name+" is not a known clusterer type:ismart, tcrdist, tcrdist_orig, tcrvalid")
 



### Wrapper function to benchmark TCR clustering on a range of reference TCR-antigen sets
- cluster methods: ismart, tcrdist, tcrvalid
- TCR-antigen references: tcrvalid internal (quality>0), GLIPH2 reference and GLIPH2_HQ (quality>0)
- multiple: 0-5x are the spike in multiples used for spike in with irrelevant TCRs for spike in robustness benchmarking.
- eps: Radius parameter, either eps for DBscan based methods or threshold parameter for ismart.
- minsize: the cluster minsize

In [14]:
def spike_score(cluster_method,reference,multiple,eps,minsize=2,spikein_hash=True):
    """ spike in multiple aka 1x, 2x etc
    """
    df_score = pd.DataFrame()
    if reference=='tcrvalid':
        df_ref = get_data('TRB', False)
    elif reference=='gliph':
        df_ref=prep_gliph_data('TRB','ref')
    elif reference=='gliph_HQ':
        df_ref_orig=prep_gliph_data('TRB','ref')
        two_up = up(up(os.getcwd()))  
        path='/tcrvalid/data/VDJDB_TCRs_w_quality.tsv'
        VDJDB_ref=pd.read_csv(two_up+path,sep='\t')
        VDJDB_ref['v_gene_TRB'] = VDJDB_ref['V'].str.split('*').str[0]
        VDJDB_ref['j_gene_TRB'] = VDJDB_ref['J'].str.split('*').str[0]
        VDJDB_ref=VDJDB_ref.rename(columns={'CDR3':'cdr3_TRB','Epitope':'peptide'}) 
        tmp_df2=df_ref_orig.merge(VDJDB_ref,on=['v_gene_TRB','cdr3_TRB','j_gene_TRB','peptide'],how='left')
        Filtered=tmp_df2.loc[(tmp_df2["Gene"]=='TRB') & (tmp_df2["Score"]>0),]                       
        Filtered=Filtered[[ 'sequence_id','cdr3_TRB', 'v_call', 'j_call', 'peptide','Score', 'source', 'v_gene_TRB', 'j_gene_TRB', 'new_v_call', 'cdr1_no_gaps', 'cdr2_no_gaps', 'cdr1_cdr2_no_gaps', 'clono_id', 'v_gene_TRA', 'j_gene_TRA', 'cdr3_TRA']]
        Filtered=Filtered.sort_values(by=['clono_id','Score'], ascending=False)
        df_ref=Filtered.drop_duplicates('clono_id')
    
    ##Path to the folder with your spike splits
    if multiple==0:
        spike_score=df_ref
        df_cl=get_clusterer_by_name(cluster_method,spike_score,eps)
        eps_format=':.2f'
        eps_name = str('eps_{'+eps_format+'}').format(eps)
        df_cl['freq_count']=df_cl.groupby('cluster')['cluster'].transform('count')
        df_cl3=df_cl.copy()
        df_cl3.loc[df_cl3.freq_count==2,'cluster']=-1
            
        #score for cluster sizes 2    
        score2=clustering_scoring(
                spike_score,
                df_cl, 
                epitope_col='peptide',
                cluster_col='cluster'
        )[0]
        #score for cluster sizes 3   

        score3=clustering_scoring(
                spike_score,
                df_cl3, 
                epitope_col='peptide',
                cluster_col='cluster'
        )[0]
        #where i is a loop variable
        df_score2=pd.DataFrame(score2,index=[cluster_method])
        df_score2['method']=df_score2.index
        df_score2=df_score2.reset_index()
        df_score2['eps']=eps_name.split('_')[1]
        df_score2['minsize']=2
        df_score2['spike_x']=multiple
        df_score2['label_reference']=reference
        df_score2['fold']=0

        
        df_score3=pd.DataFrame(score3,index=[cluster_method])
        df_score3['method']=df_score3.index
        df_score3=df_score3.reset_index()
        df_score3['eps']=eps_name.split('_')[1]
        df_score3['minsize']=3
        df_score3['spike_x']=multiple
        df_score3['label_reference']=reference
        df_score3['fold']=0
        
        df_score=pd.concat([df_score2,df_score3])


        
        

    else:
        base_name='S'+str(multiple)
        two_up = up(up(os.getcwd()))    
        path='/tcrvalid/data/Spike_splits/'
        #path='/data/home/allen.leary/repos/tcrvalid/tcrvalid/data/Spike_splits/'
        for x in range(1, 11):
            import_name=two_up+path+base_name+'_'+str(x)+'.csv'
            #import_name=path+base_name+'_'+str(x)+'.csv'
            df_merge=prep_gliph_data('TRB',import_name)
            if spikein_hash==True:
                df_merge['peptide']=pd.util.hash_pandas_object(df_merge).astype(str)
            else:
                df_merge['peptide']=np.nan
                    
            df_merge['spike_fold']=x
            
            #Merge the antigen labelled reference with the spike in TCRs
            spike_score=pd.concat([df_ref,df_merge],join='inner', ignore_index=True)    
            df_cl=get_clusterer_by_name(cluster_method,spike_score,eps)
            eps_format=':.2f'
            eps_name = str('eps_{'+eps_format+'}').format(eps)
            df_cl['freq_count']=df_cl.groupby('cluster')['cluster'].transform('count')
            df_cl3=df_cl.copy()
            df_cl3.loc[df_cl3.freq_count==2,'cluster']=-1
            #df_cl.loc[df_cl.freq_count<minsize,'cluster']=-1

            score2=clustering_scoring(
                spike_score,
                df_cl, 
                epitope_col='peptide',
                cluster_col='cluster')[0]
            score3=clustering_scoring(
                spike_score,
                df_cl3, 
                epitope_col='peptide',
                cluster_col='cluster'
        )[0]
            
            df_score2= pd.DataFrame(score2,index=[x])
            df_score2['fold']=df_score2.index
            df_score2['method']=cluster_method
            df_score2['eps']=eps_name.split('_')[1]
            df_score2['minsize']=2
            df_score2['spike_x']=multiple
            df_score2['label_reference']=reference
            
            df_score3=pd.DataFrame(score3,index=[x])
            df_score3['fold']=df_score3.index
            df_score3['method']=cluster_method
            df_score3=df_score3.reset_index()
            df_score3['eps']=eps_name.split('_')[1]
            df_score3['minsize']=3
            df_score3['spike_x']=multiple
            df_score3['label_reference']=reference 
            
            df_score_int=pd.concat([df_score2,df_score3])
            df_score=df_score.append(df_score_int)
    
    
    #directory_name='/data/home/allen.leary/repos/tcrvalid/comparitor_tooling/Spikein_scores/'+cluster_method+'reference_'+reference
    #print(directory_name)
    #Path(directory_name).mkdir(parents=True, exist_ok=True)
    #save_name='/data/home/allen.leary/repos/tcrvalid/comparitor_tooling/Spikein_scores/'+cluster_method+'reference_'+reference+'/Spikein'+str(multiple)+str(eps_name)+'.csv'
    #print(save_name)
    #df_score.to_csv(save_name)
    return df_score


### Comprehensive loop to generate scoring master csv
- This loop computes all spike in folds for smart, tcrvalid and tcrdist on both tcrvalid internal reference and gliph TCR-Antigen references.
- Also current iteration only saves at the very end, can uncomment last lines of spike score function to recreate subfolders with granular scoring output

In [ ]:

multiple_range=[0,1,2,3,4,5]
cluster_tools=['ismart','tcrvalid','tcrdist']
references=['tcrvalid','gliph']
eps_range_per_method=[np.linspace(5.5,8,6),np.linspace(0.6,6,20),np.concatenate((np.linspace(10,30,11),np.linspace(34,50,5)),axis=0)]

Clustering_df=pd.DataFrame()

for count,cluster_tool in enumerate(cluster_tools):
    eps_range=eps_range_per_method[count]
    for reference in references:
        for multiples in multiple_range:
            for eps in eps_range:
                temp_df=spike_score(cluster_method=cluster_tool,minsize=2,reference=reference,multiple=multiples,eps=eps)
                Clustering_df=Clustering_df.append(temp_df)
                print(str(cluster_tool)+'ref_'+str(reference)+'spike'+str(multiples)+'x'+'eps'+str(eps))
                print(len(Clustering_df))
Clustering_df.to_csv('cluster_loop_out.csv')    

run :  source /data/miniconda3/etc/profile.d/conda.sh && conda activate ismart && python /data/home/allen.leary/repos/tcrvalid/comparitor_tooling/distance_based_tools/tcrclustering/../scripts/modified_third_party/ismart/iSMARTv3.py -f /data/home/allen.leary/tmp/tcrclustering/tmp_ismart_data.txt -o /data/home/allen.leary/tmp/tcrclustering_output/ -t 5.5 -g -6 -n 1 --KeepPairwiseMatrix
ismartref_tcrvalidspike0xeps5.5
2
run :  source /data/miniconda3/etc/profile.d/conda.sh && conda activate ismart && python /data/home/allen.leary/repos/tcrvalid/comparitor_tooling/distance_based_tools/tcrclustering/../scripts/modified_third_party/ismart/iSMARTv3.py -f /data/home/allen.leary/tmp/tcrclustering/tmp_ismart_data.txt -o /data/home/allen.leary/tmp/tcrclustering_output/ -t 6.0 -g -6 -n 1 --KeepPairwiseMatrix
ismartref_tcrvalidspike0xeps6.0
4
run :  source /data/miniconda3/etc/profile.d/conda.sh && conda activate ismart && python /data/home/allen.leary/repos/tcrvalid/comparitor_tooling/distance_bas